In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [2]:
import numpy as np
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [90]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(130),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(130),
        transforms.CenterCrop(130),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [91]:
data_dir = 'data'

In [92]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [93]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test']}

In [94]:
dataset_sizes  = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 793, 'test': 207}

In [95]:
class_names = image_datasets['train'].classes
class_names

['cavalo',
 'comida',
 'dinossauro',
 'elefante',
 'flor',
 'indio',
 'montanha',
 'monumento',
 'onibus',
 'praia']

In [104]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(3, 16, 3)  #130 -> 64
        self.conv1 = nn.Conv2d(16, 32, 3) #62 -> 31
        self.conv2 = nn.Conv2d(32, 32, 3) #29 -> 14
        self.conv3 = nn.Conv2d(32, 64, 5) #10 -> 5        
        self.fc1 = nn.Linear(64 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)

        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv0(x), 2))
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [105]:
model = Net()

In [106]:
criterion = nn.CrossEntropyLoss()

In [107]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [108]:
def train(model, dataloader, optimizer, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [109]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [110]:
model.to(device);

In [111]:
for epoch in range(1, 50+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['test'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.9

Train Epoch: 1 [0/793 (0%)]	Loss: 2.307311
Train Epoch: 1 [640/793 (80%)]	Loss: 2.147467

Test set: Average loss: 0.0696, Accuracy: 35/207 (17%)

Train Epoch: 2 [0/793 (0%)]	Loss: 2.124352
Train Epoch: 2 [640/793 (80%)]	Loss: 1.883727

Test set: Average loss: 0.0636, Accuracy: 40/207 (19%)

Train Epoch: 3 [0/793 (0%)]	Loss: 2.043053
Train Epoch: 3 [640/793 (80%)]	Loss: 1.822283

Test set: Average loss: 0.0584, Accuracy: 60/207 (29%)

Train Epoch: 4 [0/793 (0%)]	Loss: 1.788349
Train Epoch: 4 [640/793 (80%)]	Loss: 1.892237

Test set: Average loss: 0.0556, Accuracy: 83/207 (40%)

Train Epoch: 5 [0/793 (0%)]	Loss: 1.837953
Train Epoch: 5 [640/793 (80%)]	Loss: 1.561149

Test set: Average loss: 0.0495, Accuracy: 100/207 (48%)

Train Epoch: 6 [0/793 (0%)]	Loss: 1.302571
Train Epoch: 6 [640/793 (80%)]	Loss: 2.050416

Test set: Average loss: 0.0514, Accuracy: 99/207 (48%)

Train Epoch: 7 [0/793 (0%)]	Loss: 1.597444
Train Epoch: 7 [640/793 (80%)]	Loss: 1.885993

Test set: Average loss: 0.0486, A

KeyboardInterrupt: 